In [186]:
epsilon = 0.5
import sys 
sys.path.append('../')

In [187]:
from sklearn.datasets._samples_generator import make_blobs
from Helpers.twod_laplace import generate_truncated_laplace_noise
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from art.estimators.classification import SklearnClassifier
from art.attacks.inference.membership_inference import LabelOnlyDecisionBoundary

X, y_true = make_blobs(n_samples=500, centers=4, n_features=2,
                       cluster_std=0.60, random_state=0)



In [188]:
from art.metrics.metrics import adversarial_accuracy, SUPPORTED_METHODS
from art.metrics.privacy import PDTP
from art.attacks.evasion import HopSkipJump
from sklearn.metrics import log_loss, roc_curve
# def create_dataset()
def run_semi_supervised_setup(data, epsilon): 
    X_scaled = pd.DataFrame(StandardScaler().fit_transform(data), columns=['X', 'Y'])
    ap = KMeans(n_clusters=4).fit(X_scaled)
    _, _, Z = generate_truncated_laplace_noise(data, epsilon)
    Z_pd = pd.DataFrame(Z, columns=['X', 'Y'])

    X_train, X_test, y_train, y_test = train_test_split(Z_pd, ap.labels_, random_state=0)
    fitted_classifier = GaussianNB().fit(X_train.values, y_train)
    return fitted_classifier.score(X_test, y_test), fitted_classifier, X_train, X_test, y_train, y_test
    #return ap

def run_adversial_advantage_private(initial_data, epsilon):
    #score, fitted_classifier = run_semi_supervised_setup(X_pd)
    score, fitted_classifier_private, X_train, X_test, y_train, y_test = run_semi_supervised_setup(initial_data, epsilon)
    print(score)
    art_model = SklearnClassifier(model=fitted_classifier_private,  clip_values=(0, initial_data.size))
    #art_model_unfitted = SklearnClassifier(model=GaussianNB())
    return run_attack(art_model, X_test, y_test)
    #return adversarial_accuracy(art_model, X_test.values,  y_test, attack_name='hsj', attack_params={ "init_eval": 10, "max_eval": 1000, "max_iter": 0 })
    #return PDTP(art_model, art_model_unfitted, X_train.values, y_train)

def run_attack(art_model, X_test, y_test):
    mia_labelonly = LabelOnlyDecisionBoundary(art_model)
    mia_labelonly.calibrate_distance_threshold_unsupervised(top_t=50, num_samples=len(X_test), max_queries=2, verbose=True, batch_size=256)
    y_pred = mia_labelonly.infer(X_test.values, y_test)
   #mia_label_only_unsupervised = LabelOnlyDecisionBoundary(art_model)
    #inferred_y = mia_label_only_unsupervised.infer(X_test, y_test)
     # Predict the labels for adversarial examples
    y_orig = np.argmax(art_model.predict(X_test), axis=1)
    #y_pred = np.argmax(art_model.predict(adv_x), axis=1)
    #print(y_orig, y_pred)

    if len(y_test.shape) > 1:
        y_test = np.argmax(y_test, axis=1)
    y_corr = y_orig == y_test
    return np.sum((y_pred == y_orig) & y_corr) / np.sum(y_corr)
    #return accuracy_score(y_test, y_pred), accuracy_score(y_test, y_orig)

def membership_inference(true_y, pred_y, membership, train_loss):
    print('-' * 10 + 'MEMBERSHIP INFERENCE' + '-' * 10 + '\n')
    pred_membership = np.where(log_loss(true_y, pred_y) <= train_loss, 1, 0)
    fpr, tpr, thresholds = roc_curve(membership, pred_membership, pos_label=1)
    print(fpr, tpr, tpr-fpr)
    mem_adv = tpr[1]-fpr[1]
    return mem_adv, log_loss(true_y, pred_y)


In [196]:
#total = []
#for n in range(0, 10):
def run_report():
    epsilons = [0.05, 0.1, 0.5, 1, 3, 5, 10, 15]
    dictionary = {}
    for epsilon in epsilons:
        data = run_adversial_advantage_private(X, epsilon)
        dictionary[epsilon] = data

    return dictionary
#print(np.average(total))
#advantage_adv, advantage = run_adversial_advantage(X)

In [197]:
print(run_report())

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xffff37387d30>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xffff37387d30>
Traceback 

0.232


/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


HopSkipJump:   0%|          | 0/125 [00:00<?, ?it/s]

HopSkipJump:   0%|          | 0/125 [00:00<?, ?it/s]

...

1.0


/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


HopSkipJump:   0%|          | 0/125 [00:00<?, ?it/s]

HopSkipJump:   0%|          | 0/125 [00:00<?, ?it/s]

{0.05: 0.2413793103448276, 0.1: 0.5151515151515151, 0.5: 0.2835820895522388, 1: 0.24489795918367346, 3: 0.2727272727272727, 5: 0.304, 10: 0.168, 15: 0.168}


In [190]:
# number of samples used to calibrate distance threshold
"""attack_train_size = 1500
attack_test_size = 1500

x = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])
training_sample = np.array([1] * len(X_train) + [0] * len(X_test))"""

'attack_train_size = 1500\nattack_test_size = 1500\n\nx = np.concatenate([X_train, X_test])\ny = np.concatenate([y_train, y_test])\ntraining_sample = np.array([1] * len(X_train) + [0] * len(X_test))'

In [191]:
"""n = 375
eval_data_idx = np.random.choice(len(x), n)
x_eval, y_eval = x[eval_data_idx], y[eval_data_idx]
eval_label = training_sample[eval_data_idx] """

'n = 375\neval_data_idx = np.random.choice(len(x), n)\nx_eval, y_eval = x[eval_data_idx], y[eval_data_idx]\neval_label = training_sample[eval_data_idx] '

In [192]:
#from art.attacks.inference.membership_inference import LabelOnlyDecisionBoundary

#mia_label_only_unsupervised = LabelOnlyDecisionBoundary(art_model,)


In [193]:
#mia_label_only_unsupervised.calibrate_distance_threshold_unsupervised(top_t=50, num_samples=n, max_queries=10, verbose=True, batch_size=256)

In [194]:

#pred_label_unsupervised = mia_label_only_unsupervised.infer(x_eval, y_eval)
#pred_label_unsupervised